In [2]:
import transformers
import time
import requests

import torch
from PIL import Image
from skimage.io import imread
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
print(transformers.__version__)
model_id = "IDEA-Research/grounding-dino-base"
device = "cuda"

processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)

4.44.2


Could not load the custom kernel for multi-scale deformable attention: /home/daoan/.cache/torch_extensions/py310_cu118/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Could not load the custom kernel for multi-scale deformable attention: /home/daoan/.cache/torch_extensions/py310_cu118/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Could not load the custom kernel for multi-scale deformable attention: /home/daoan/.cache/torch_extensions/py310_cu118/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Could not load the custom kernel for multi-scale deformable attention: /home/daoan/.cache/torch_extensions/py310_cu118/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Could not load the custom kernel for

In [2]:
import os
import re


def find_all_keyframe_files(root_dir, extensions=[".jpg"]):
    files = []
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(filename.lower().endswith(ext) for ext in extensions):
                files.append(os.path.join(dirpath, filename))
    return files

In [3]:
def get_video_id_and_frame_id(file_path):
    pattern = r"/([A-Z0-9]+)_extra/([A-Z0-9]+)/(\d+)\.*"
    match = re.search(pattern, file_path)
    if match:
        video_id = f"{match.group(1)}_{match.group(2)}"
        frame_id = match.group(3)
        return video_id, frame_id
    else:
        print("No match found: " + file_path)
        return None, None

In [21]:
def get_tags_from_keyframe_file(file_path):
    with open(file_path.replace("Keyframes", "Tags").replace("jpg", "txt"), "r") as file:
        return file.read().replace("|", ".").strip() + " . "

In [393]:
def get_od_from_keyframe_file(file_path):
    od_path = file_path.replace("Keyframes", "object_detection").replace("jpg", "txt")
        # create folder if not exists
    if not os.path.exists(os.path.dirname(od_path)):
        os.makedirs(os.path.dirname(od_path))
    return od_path

In [54]:
def split_list_into_chunks(lst, chunk_size):
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

In [40]:
key_frame_paths = find_all_keyframe_files("/media/daoan/T7 Shield2/VN_Multi_User_Video_Search/Keyframes")
keywords = ["L16_extra", "L17_extra", "L18_extra", "L19_extra", "L20_extra", "L21_extra"]
key_frame_paths = [path for path in key_frame_paths if any(keyword in path for keyword in keywords)]
len(key_frame_paths)

1383856


In [394]:
from tqdm import tqdm

# Define the batch size
batch_size = 4

# List of image paths
chucks = split_list_into_chunks(key_frame_paths, batch_size)

for chuck in tqdm(chucks, "Processing batches"):
# Load and preprocess images in batches
    batch_images = []
    for image_path in chuck:
        image = Image.open(image_path).convert("RGB")
        batch_images.append(image)
    texts = []
    for image_path in chuck:
        texts.append(get_tags_from_keyframe_file(image_path))

    inputs = processor(images=batch_images, text=texts, return_tensors="pt",
                       padding=True,
                       truncation=True
                       ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    # Post-process the results for each batch
    results = processor.post_process_grounded_object_detection(
        outputs,
        inputs.input_ids,
        box_threshold=0.4,
        text_threshold=0.3,
        target_sizes=[image.size[::-1] for image in batch_images]
    )
    
    for index, result in enumerate(results):
        scores = result['scores']
        labels = result['labels']
        boxes = result['boxes']
        data = []
        for score, label, box in zip(scores, labels, boxes):
            box = box.tolist()
            data.append(f"{label} | {score:.2f} | {box[0]} {box[1]} {box[2]} {box[3]}")       
        with open(get_od_from_keyframe_file(chuck[index]), "w", encoding='utf-8') as file:
            file.write("\n".join(data))

Processing batches: 100%|██████████| 345964/345964 [53:32:41<00:00,  1.79it/s]   
